# Inversion of ERT transects

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from resipy import Project

ERTdatadir = 'data/ERT/'

In [2]:
# Parameters
extract = 0.5 # 0.5 and 2.5

## Middelkerke

In [ ]:
# reading electrode position
dfm = pd.read_csv(ERTdatadir + 'electrode_locations_Middelkerke.csv')
elec = np.zeros((120, 3))
elec[:, 0] = dfm['distance'].values
dfm.head()

In [ ]:
# invert
km = Project()
km.createSurvey(ERTdatadir + '23082301.csv', ftype='Syscal')
km.setElec(elec)
km.filterAppResist(vmin=0)
km.showPseudo()
km.filterRecip() 
km.fitErrorPwl()
km.createMesh('trian')
km.showMesh()

In [ ]:
# invert
km.err = True  # use fitted error in the inversion
km.invert()

In [ ]:
# inverted transect
fig, ax = plt.subplots(figsize=(10, 4))
km.showResults(ax=ax, sens=False, attr='Resistivity(ohm.m)', vmin=0, vmax=20)

fig.savefig(ERTdatadir + 'middelkerke-inv-ERT-'+'.jpg', dpi=300)

In [44]:
# extract profiles
m_m = km.meshResults[0]

dfs = []
for i in range(dfm.shape[0]):
    row = dfm.loc[i, :]
    ie = m_m.df['X'].between(row['distance'] - extract, row['distance'] + extract) & m_m.df['Z'].gt(-5) # 0.5 and 2.5
    sdf = m_m.df[ie][['Z', 'Resistivity(ohm.m)']]
    sdf['Z'] = sdf['Z'].round(1)
    #sdf['Z'] = (sdf['Z'] * 2).round(1) / 2
    sdf = sdf.groupby('Z').mean().reset_index()
    sdf['easting'] = row['easting']
    sdf['northing'] = row['northing']
    sdf['ID'] = row['ID']
    dfs.append(sdf)
dfmi = pd.concat(dfs)

dfmi.to_csv(ERTdatadir + 'middelkerke-inv-ERT-'+str(extract)+'.csv', index=False)

In [ ]:
# figure of profiles
fig, ax = plt.subplots()
for el in dfm['ID'].unique():
    ie = dfm['ID'].eq(el)
    ax.plot(dfmi[ie]['Resistivity(ohm.m)'], dfmi[ie]['Z'], '-')
ax.set_xlabel('Resistivity [Ohm.m]')
ax.set_ylabel('Z [m]')
fig.savefig(ERTdatadir + 'middelkerke-prof'+str(extract)+'.jpg')

## Proefhoeve

In [ ]:
# reading electrode position
dfp = pd.read_csv(ERTdatadir + 'electrode_locations_Proefhoeve.csv')
dfp = dfp[:-1]
elec = np.zeros((60, 3))
elec[:, 0] = dfp['distance'].values
dfp.head()

In [ ]:
kp = Project()
kp.createSurvey(ERTdatadir + '23082201.csv', ftype='Syscal')
kp.setElec(elec)
kp.filterAppResist(vmin=0)
kp.showPseudo()
kp.filterRecip() 
kp.fitErrorPwl()
kp.createMesh('trian')
kp.showMesh()

In [ ]:
# invert
kp.err = True  # use fitted error in the inversion
kp.invert()

In [ ]:
# inverted transect
fig, ax = plt.subplots(figsize=(10, 4))
kp.showResults(ax=ax, attr='Resistivity(ohm.m)', sens=False, vmin=4, vmax=40)

fig.savefig(ERTdatadir + 'proefhoeve-inv-ERT-'+'.jpg', dpi=300)

In [47]:
# extract profiles
mp = kp.meshResults[0]
pd.set_option('display.max_rows', None)

dfsp = []
for i in range(dfp.shape[0]):
    row = dfp.loc[i, :] # Each row of the original ERT positions
    mask = mp.df['X'].between(row['distance'] - extract, row['distance'] + extract) & mp.df['Z'].gt(-5) # Mask for each element of the ERT 
    sdf = mp.df[mask][['Z', 'Resistivity(ohm.m)']] # Depth and resistivity of the filtered elements
    sdf['Z'] = sdf['Z'].round(1)
    sdf = sdf.groupby('Z').mean().reset_index() # Take the mean
    sdf['easting'] = row['easting']
    sdf['northing'] = row['northing']
    sdf['ID'] = row['ID'] # After taking the mean, each profile is set in the original ID
    dfsp.append(sdf)
dfpi = pd.concat(dfsp)
dfpi.to_csv(ERTdatadir + 'proefhoeve-inv-ERT-'+str(extract)+'.csv', index=False)

In [ ]:
# figure of profiles
fig, ax = plt.subplots()
for el in dfpi['ID'].unique():
    ie = dfpi['ID'].eq(el)
    ax.plot(dfpi[ie]['Resistivity(ohm.m)'], dfpi[ie]['Z'], '-')
ax.set_xlabel('Resistivity [Ohm.m]')
ax.set_ylabel('Z [m]')
fig.savefig(ERTdatadir + 'proefhoeve-prof'+str(extract)+'.jpg')